# Monad Types in JavaScript

Based on [Understanding Monads](https://en.wikibooks.org/wiki/Haskell/Understanding_monads) in Haskell.

A monad is defined by three things:

* a [type constructor](https://en.wikibooks.org/wiki/Haskell/More_on_datatypes#Parameterized_Types) `m` (does not exist in JavaScript)
* a function `return`
* an operator `(>>=)` which is pronounced "bind"

The function and operator are methods of the Monad class:

In [1]:
const T = Symbol('Type')

/**
 * Abstract base class Monad[T]
 *
 * We use ES2015 classes to declare an 'abstract' base type. Since there
 * are no abstract classes in JavaScript, we must rely on the user to
 * only instantiate the proper subclasses through the provided methods.
 */
class Monad {
    /**
     * @param {*} x - The actual value wrapped by the Monad
     * @param {function} t - The type of the concrete subclass
     */
    constructor(x, t) {
        this.x = x
        this[T] = t
    }

    /**
     * return :: (Monad m) => a -> m a
     *
     * In principle the constructor already defines the 'return' (or 'unit')
     * function. However because it requires the use of 'new' and cannot
     * be called as a regular function, we use this implementation instead.
     *
     * We must declare this method as 'static' so that it can be used
     * without already having a Monad instance.
     *
     * @param {*} x - A raw value that will be wrapped inside the Monad
     * @returns {Monad} - An instance of a concrete Monad subclass
     */
    static return(x) {
        throw new Error('derived types must implement return') 
    }

    /**
     * bind :: (Monad m) => m a -> (a -> m b) -> m b
     *
     * The purpose of this method is to transform the Monad
     * into another Monad by applying a special function.
     *
     * @abstract
     * @param {function} g - Function of type (a -> m b)
     * @returns {Monad} - The result of applying g to this Monad
     */
    bind(g) {
        throw new Error('derived types must implement bind')
    }

    /**
     * then :: (Monad m) => m a -> m b -> m b
     * 
     * A convenience method that directly binds another
     * Monad without performing any transformation. The
     * result still depends on the implementation of bind.
     * 
     * @param {Monad} m
     * @returns {Monad} - The result of binding m to this Monad
     */
    then(m) {
        return this.bind(_ => m)
    }

    /**
     * liftM :: m a1 -> (a1 -> r) -> m r
     *
     * Like bind, but we can apply a function of type (a1 -> r)
     *
     * liftM (or fmap) is typically written as (a1 -> r) -> m a1 -> m r
     * However, because we must declare it as a method of this class, we
     * reverse the order of the first two statements.
     *
     * The only way to provide a default implementation of liftM
     * in this base class is by having a reference to the right
     * implementation of 'return'. Thus we employ the following trick:
     * When instantiating the concrete subclass we pass the subclass
     * type to this abstract base class in the constructor.
     * Now we have access to the static methods of that subclass.
     *
     * @returns {Monad}
     */
    liftM(f) {
        return this.bind(x => this[T].return(f(x)))
    }
}

[Function: Monad]

## Maybe Monad

In [2]:
class Maybe extends Monad {
    /**
     * bind :: Maybe a -> (Maybe -> Maybe b) -> Maybe b
     * @override
     * @param {function} g
     * @returns {Maybe}
     */
    bind(g) {
        // In other languages we could use pattern matching instead.
        if (this instanceof Just)
            return g(this.x)
        else
            return Nothing.return()
    }
}

class Just extends Maybe {
    /**
     * return :: a -> Just a
     * @override
     * @param {*} x
     * @returns {Just}
     */
    static return(x) {
        return new Just(x, Just)
    }
}

class Nothing extends Maybe {
    /**
     * return :: Nothing
     * @override
     * @returns {Nothing}
     */
    static return() {
        return new Nothing(null, Nothing)
    }
}

[Function: Nothing]

## Maybe examples

In [3]:
Just.return(1)
    .bind(x => Just.return(x + 1))
    .bind(x => Just.return(x + 1))
    .liftM(x => x * x)

Just { x: 9 }

### Save division

In [4]:
function divide(d) {
    return (x => {
        if (Number.isFinite(x) && Number.isFinite(d) && d !== 0)
            return Just.return(x / d)
        else
            return Nothing.return()
    })
}

undefined

In [5]:
Just.return(32)
    .bind(divide(2))
    .liftM(Math.sqrt)

Just { x: 4 }

In [6]:
Just.return(32)
    .bind(divide(NaN))
    .liftM(Math.sqrt)

Nothing { x: null }

### Safe properties

In [7]:
function get(property) {
    return (object => {
        if (object.hasOwnProperty(property))
            return Just.return(object[property])
        else
            return Nothing.return()
    })
}

undefined

In [8]:
Just.return({foo: {bar: 42}})
    .bind(get('foo'))
    .bind(get('bar'))

Just { x: 42 }

In [9]:
Just.return({foo: {bar: 42}})
    .bind(get('baz'))
    .bind(get('bar'))

Nothing { x: null }

## Monad Laws

Every instance of the Monad class (and thus all implementations of bind (>>=) and return) must obey the following three laws:

```haskell
m >>= return     =  m                        -- right unit
return x >>= f   =  f x                      -- left unit

(m >>= f) >>= g  =  m >>= (\x -> f x >>= g)  -- associativity
```

In [10]:
(function() {
    const equal = require('assert').deepEqual

    const ret = Just.return
    const m = ret(42)
    const f = (x => ret(x * x))
    const g = (x => ret(-x))

    // right unit
    equal(m.bind(ret), m)

    // left unit
    equal(ret(42).bind(f), f(42))

    // associativity
    equal(m.bind(f).bind(g), m.bind(x => f(x).bind(g)))

    return 'ok'
}())

'ok'

(node:367) Warning: Possible EventEmitter memory leak detected. 11 unpipe listeners added. Use emitter.setMaxListeners() to increase limit
(node:367) Warning: Possible EventEmitter memory leak detected. 11 drain listeners added. Use emitter.setMaxListeners() to increase limit
(node:367) Warning: Possible EventEmitter memory leak detected. 11 error listeners added. Use emitter.setMaxListeners() to increase limit
(node:367) Warning: Possible EventEmitter memory leak detected. 11 close listeners added. Use emitter.setMaxListeners() to increase limit
(node:367) Warning: Possible EventEmitter memory leak detected. 11 finish listeners added. Use emitter.setMaxListeners() to increase limit
(node:367) Warning: Possible EventEmitter memory leak detected. 11 unpipe listeners added. Use emitter.setMaxListeners() to increase limit
(node:367) Warning: Possible EventEmitter memory leak detected. 11 drain listeners added. Use emitter.setMaxListeners() to increase limit
(node:367) Warning: Possible Ev

## Identity Monad

In [11]:
class Identity extends Monad {
  static return (x) {
    return new Identity(x, Identity)
  }

  bind(g) {
    return g(this.x)
  }
}

[Function: Identity]

## Continuation Monad

In [12]:
class Cont extends Monad {
  static return (y) {
    return new Cont(f => f(y), Cont)
  }

  bind(g) {
    return this.x(g)
  }
}

[Function: Cont]

In [13]:
Cont.return(42)
  .bind(x => Cont.return(x + 1))
  .bind(x => Cont.return(x + 1))
  .bind(Identity.return)

Identity { x: 44 }

## List Monad

In [14]:
class List extends Monad {
  static return (x) {
    return new List([x], List)
  }

  bind(g) {
    return this.x.map(g).reduce((xs, x) => xs.concat(x))
  }
}

[Function: List]

In [15]:
List.return(2).liftM(x => x*x)

List { x: [ 4 ] }